In [31]:
# AI-Powered Web Summarizer

# A program that looks at any webpage on the internet, 
# scrape the content of the webpage and summarize it and 
# present a short summary of the webpage

In [32]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup # for web scraping (web parsing)
from openai import OpenAI
from IPython.display import Markdown, display

In [33]:
# Load environment variables in a file .env

load_dotenv()
openai_key=os.getenv("OPENAI_API_KEY")

# Check the key
if not openai_key:
    print("An API key was not found")
elif openai_key[:8]!="sk-proj-":
    print("An API key was found, but it dosen't start with sk-proj-")
elif openai_key.strip()!= openai_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them")
else:
    print("An API key was found and looks good")

An API key was found and looks good


In [34]:
# Create an instance of openai

openai = OpenAI()

In [35]:
# A class to represent a Webpage

class Website:
    """
    A utility calss to represent a website that we have scraped
    """
    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeutifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [36]:
# output

site = Website("https://fulafia.edu.ng")
print(site.title)
print(site.text)

Federal University of Lafia – The official website of the Federal University of Lafia (FULafia), providing access to academic programs, research, admissions, faculties, and resources for students, staff, and the community
Federal University of Lafia
info@fulafia.edu.ng
+234 811 299 0109
About Us
Academics
Faculty of Computing
Faculty of Law
Faculty of Management Science
Faculty of Medical Laboratory Science
Faculty of Nursing
Faculty of Pharmaceutical Science
Faculty of Education
Faculty of Radiography and Applied Health Sciences
Faculty of Physical Sciences
Faculty of Basic Clinical Sciences
Faculty of Basic Medical Sciences
Faculty of Clinical Sciences
Faculty of Engineering
Faculty of Environmental Design
Faculty of Arts
Faculty of Agriculture
Faculty of Veterinary
Faculty of Social Science
Faculty of Life Sciences
College of Health Sciences
College of Medicine
College of Postgraduate Studies
School of Basic and Remedial Studies
Units
Library
Legal Unit (Registry)
Registry
Internal 

In [37]:
# Define our system prompt

system_prompt = "You are an assistant that analyzes the contents of a website" \
"and provides a short summary, ignoring text that might be navigation related." \
"Respond in markdown."

In [38]:
# A function that writes a User Prompt that asks for summaries of websites

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt +="\nThe contents of this website is as follows:"
    "please provide a short summary of this website in markdown."
    "If it includes news or announcement, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [39]:
print(user_prompt_for(site))

You are looking at a website titled Federal University of Lafia – The official website of the Federal University of Lafia (FULafia), providing access to academic programs, research, admissions, faculties, and resources for students, staff, and the community
The contents of this website is as follows:Federal University of Lafia
info@fulafia.edu.ng
+234 811 299 0109
About Us
Academics
Faculty of Computing
Faculty of Law
Faculty of Management Science
Faculty of Medical Laboratory Science
Faculty of Nursing
Faculty of Pharmaceutical Science
Faculty of Education
Faculty of Radiography and Applied Health Sciences
Faculty of Physical Sciences
Faculty of Basic Clinical Sciences
Faculty of Basic Medical Sciences
Faculty of Clinical Sciences
Faculty of Engineering
Faculty of Environmental Design
Faculty of Arts
Faculty of Agriculture
Faculty of Veterinary
Faculty of Social Science
Faculty of Life Sciences
College of Health Sciences
College of Medicine
College of Postgraduate Studies
School of Ba

In [40]:
def messages_for(website):
    return [
        {"role":"system", "content": system_prompt},
        {"role":"user", "content":user_prompt_for(website)}
    ]

In [41]:
messages_for(site)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a websiteand provides a short summary, ignoring text that might be navigation related.Respond in markdown.'},
 {'role': 'user',
  'content': "You are looking at a website titled Federal University of Lafia – The official website of the Federal University of Lafia (FULafia), providing access to academic programs, research, admissions, faculties, and resources for students, staff, and the community\nThe contents of this website is as follows:Federal University of Lafia\ninfo@fulafia.edu.ng\n+234 811 299 0109\nAbout Us\nAcademics\nFaculty of Computing\nFaculty of Law\nFaculty of Management Science\nFaculty of Medical Laboratory Science\nFaculty of Nursing\nFaculty of Pharmaceutical Science\nFaculty of Education\nFaculty of Radiography and Applied Health Sciences\nFaculty of Physical Sciences\nFaculty of Basic Clinical Sciences\nFaculty of Basic Medical Sciences\nFaculty of Clinical Sciences\nFaculty of En

In [42]:
# Using OpenAI API to summarize

def summarize(url):
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages_for(url)
    )
    return response.choices[0].message.content

In [43]:
summarize(site)

"# Summary of Federal University of Lafia Website\n\n**Overview**  \nThe Federal University of Lafia (FULafia), established in 2011, aims to provide access to quality higher education and is recognized for its excellence in teaching, research, and community service. Located in Lafia, Nasarawa State, Nigeria, the university emphasizes digital learning and has rapidly expanded its academic offerings and infrastructure.\n\n**Academic Structure**  \nFULafia comprises 22 faculties and offers more than 100 programs, including:\n\n- **Faculties** such as Computing, Law, Management Science, Medical Laboratory Science, Nursing, Pharmacy, Education, Engineering, and many more.\n- **Colleges**, including the College of Health Sciences and the College of Medicine.\n\n**Vision & Mission**  \nThe institution's vision is to be a world-class university recognized for excellence, with a mission focusing on fostering high standards in education, research, and service.\n\n**Campus Life**  \nThe universit

In [44]:
# A function to display this nicely in the jupyter output using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [45]:
display_summary(site)

# Summary of the Federal University of Lafia Website

The Federal University of Lafia (FULafia) is a Nigerian institution established in 2011, focused on providing quality higher education. The university is located in Lafia, Nasarawa State, and emphasizes excellence in teaching, research, and community service while supporting over 20,000 students across more than 100 academic programs.

## Key Features:
- **Faculties and Colleges**: FULafia comprises 22 faculties, including those focused on Agriculture, Arts, Computing, Law, Medical Sciences, Engineering, and Veterinary Sciences. The university also includes specialized colleges, such as the College of Health Sciences and College of Medicine.
  
- **Educational Resources**: The institution offers various resources and services like a digital library, well-equipped research labs, and modern lecture halls aimed at enhancing academic experiences.

- **Vision and Mission**: The university aims to become a world-class institution known for its contributions to teaching, research, and community service.

- **Campus Life**: Students at FULafia benefit from diverse sports and cultural activities, promoting a balanced university experience.

- **Admission and Programs**: Prospective students can apply for undergraduate and postgraduate programs through the university's online portal.

The Federal University of Lafia is committed to developing future leaders and innovators through its impressive academic offerings and community-oriented initiatives.